In [1]:
import psycopg2
from psycopg2 import sql

In [2]:
conn = psycopg2.connect(
                dbname="Logger",
                user="postgres",
                password="Subbu@2002",
                host="localhost",
                port="5432"
            )
cur = conn.cursor()

In [3]:
conn, cur

(<connection object at 0x7f7dc0810c10; dsn: 'user=postgres password=xxx dbname=Logger host=localhost port=5432', closed: 0>,
 <cursor object at 0x7f7dc0828a90; closed: 0>)

In [4]:
insert = sql.SQL("INSERT INTO logs (Job_ID, Level, Message, Timestamp) VALUES ({}, {}, {}, {})").format(
    sql.Literal("pk_12121321"),
    sql.Literal("Warning"),
    sql.Literal("Test message HI"),
    sql.Literal("2021-08-01 00:00:00")
)
cur.execute(insert)
conn.commit()

In [1]:
# Make sure to add the S3 filepath [s3://data-engg-prod/mlops/utilities/] in the job config
# Import the logs_utils
import Logger as log

# LOGS for Job Run
The following methods use an Logger object created in the class CloudWatchLogsLogger

In [6]:
# Creating the object for generating log events on Cloudwatch in the class CloudWatchLogger.
logger = log.PostgresLogger(job_id = "Logger_test", details = "Snowflake_Cost BIU", job_type = "Glue")

## Transformation

In [9]:
transformation = logger.start_transformation(name = "Image_generated", details = "Generates image and its stats", SQL_query = "SELECT * WHERE DUMMY_QUERY1")

In [10]:
logger.end_transformation(transformation, info = {"Dummy" : 89})

## Ending the job

In [11]:
# Ending the job. It maintains the job end time in the variable endtime and the total runtime in the variable totalruntime. The attribute is the logger object.
logger.end(info = {"Snowflake table it's was":'PQPL_data'})

In [16]:
import psycopg2
from psycopg2 import sql
import random
from faker import Faker
import json

# Connect to your postgres DB
conn = psycopg2.connect(
    dbname="Logger",
    user="postgres",
    password="Subbu@2002",
    host="localhost",
    port="5432"
)
cur = conn.cursor()

# Create a Faker instance
fake = Faker()

# Define the possible values for some fields
job_types = ["AWS Sagemaker", "Glue", "Other"]
statuses = ["Success", "Failure"]

# Generate and insert the random entries
for _ in range(2500):
    job_info = {
        "Job ID": fake.uuid4(),
        "Details": fake.sentence(),
        "Job Type": random.choice(job_types),
        "Start Time": str(fake.date_time_this_year()),
        "End Time": str(fake.date_time_this_year()),
        "Total Runtime": str(fake.time_delta()),
        "Info": fake.sentence(),
        "Transformations": [
            {
                "Name": fake.word(),
                "Details": fake.sentence(),
                "SQL Query": "SELECT * FROM " + fake.word(),
                "Start Time": str(fake.date_time_this_year()),
                "End Time": str(fake.date_time_this_year()),
                "Total Runtime": str(fake.time_delta()),
                "Info": {fake.word(): fake.word() for _ in range(random.randint(1, 5))}
            } for _ in range(random.randint(1, 5))
        ],
        "Status": random.choice(statuses)
    }

    insert = sql.SQL("""
        INSERT INTO stats (job_id, details, job_type, start_time, end_time, total_runtime, info, transformations) 
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
    """)
    data = (
        job_info["Job ID"],
        job_info["Details"],
        job_info["Job Type"],
        job_info["Start Time"],
        job_info["End Time"],
        job_info["Total Runtime"],
        json.dumps(job_info["Info"]),
        json.dumps(job_info["Transformations"]),
    )
    cur.execute(insert, data)

# Commit the changes and close the connection
conn.commit()
cur.close()
conn.close()